In [1]:
import pandas as pd
import statsmodels.api as sm

Warning: Runnig this model gives out warnings that we could not deal with. It may has to do with memory, or the form and shape of our current data. It will cause some tables to be filled with missing values each time you run the model. It seemed that the more you run it, the more missing values it creates.
Same for the second model in the next block.

In [ ]:
df_markets = pd.read_csv('../../01_data/02_pre/markets_info.csv')


with open('../../03_documentation/multinomial_reg_results.txt', 'w') as f:
    for ticker in df_markets['ticker']:
        df= pd.read_csv(f'../../01_data/03_analysis/{ticker}.csv')
        
        #Dealing with missing values
        df[['sum', 'max_mag', 'max_sig', 'min_depth', 'min_dist', 'tsunami']] = df[['sum', 'max_mag', 'max_sig', 'min_depth', 'min_dist', 'tsunami']].fillna(0)
        df['rev_dist'] = df['min_dist'].apply(lambda x: 0 if x == 0 else 1 / x)
        df['rev_dist_sq'] = df['rev_dist'] * df['rev_dist']
        df['rev_depth'] = df['min_depth'].apply(lambda x: 0 if x == 0 else 1 / x)
    
        #This part creates the regression model
        X = sm.add_constant(df[['num', 'max_sig', 'rev_dist']])
        y = df['effect']
        mlogit_model = sm.MNLogit(y, X)
        mlogit_model.fit(method='bfgs', maxiter=200)
        result = mlogit_model.fit()
        

        #Saving it to the file
        f.write(f"Ticker: {ticker}\n")
        f.write(result.summary().as_text())
        f.write("\n" + "="*80 + "\n")

As the model assumes that depth and distance are reversely associated with change in the market, when we assign 0 to them the model will be confused. Therefore I assigned maximum, unreal numbers, to both to see if they might matter. Turns out, they don't.

In [ ]:
df_markets = pd.read_csv('data/test/markets.csv')


with open('data/test/linear/results_logistic corrected distance and depth.txt', 'w') as f:
    for ticker in df_markets['Ticker']:
        df= pd.read_csv(f'data/test/linear/{ticker}.csv')
        
        #Dealing with missing values
        df[['sum', 'max_mag', 'max_sig', 'tsunami']] = df[['sum', 'max_mag', 'max_sig', 'tsunami']].fillna(0)
        df[['min_dist']] = df[['min_dist']].fillna(20000)
        df[['min_depth']] = df[['min_depth']].fillna(20000)
        df['rev_dist'] = df['min_dist'].apply(lambda x: 0 if x == 0 else 1 / x)
        df['rev_dist_sq'] = df['rev_dist'] * df['rev_dist']
        df['rev_depth'] = df['min_depth'].apply(lambda x: 0 if x == 0 else 1 / x)
    
    
        #This part creates the regression model
        X = sm.add_constant(df[['num', 'max_sig', 'rev_dist']])
        y = df['effect']
        mlogit_model = sm.MNLogit(y, X)
        mlogit_model.fit(method='bfgs', maxiter=200)
        result = mlogit_model.fit()
        

        #Saving it to the file
        f.write(f"Ticker: {ticker}\n")
        f.write(result.summary().as_text())
        f.write("\n" + "="*80 + "\n")